In [1]:
# Load Libraries
import warnings
import keras
import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.layers import Input, ELU, Embedding, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Embedding, GRU
from keras.layers.core import Dense, Dropout, Flatten

from model_evaluator import Evaluator
from model_preprocessor import Preprocessor

Using TensorFlow backend.


In [2]:
# LSTM Model
def gru(max_len=80, emb_dim=32, max_vocab_len=128, W_reg=regularizers.l2(1e-4)):
    """LSTM with Attention model with the Keras Sequential model"""

    input = Input(shape=(max_len,), dtype='int32', name='gru_input')
    
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(input)
    emb = Dropout(0.2)(emb)
    
    lstm = GRU(units=128, return_sequences=True)(emb)
    lstm = Dropout(0.5)(lstm)
    lstm = Flatten()(lstm)
    
    h1 = Dense(8576, activation='relu')(lstm)
    h1 = Dropout(0.5)(h1)
    
    output = Dense(1, activation='sigmoid', name='gru_output')(h1)
    
    model = Model(input=[input], output=[output])
   
    return model

In [3]:
x_train, x_test, y_train, y_test = Preprocessor.load_data_binary(10000)

                                                       url    label  class
282377   https://pmpawarbounces.duoliprudential.com.wat...  malware      1
1210677                 http://211.137.225.68:53803/Mozi.m  malware      1
240450            http://104.168.146.56:80/bins/kalon.m68k  malware      1
889081   http://ranmureed.com/sitemaps/Document/5jpoott...  malware      1
233629                  http://111.43.223.154:39690/Mozi.m  malware      1
x_train:  (8000, 80)
y_train:  (8000, 1)
x_test:  (2000, 80)
y_test:  (2000, 1)


In [4]:
epochs = 5
batch_size = 64

model = gru()
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.BinaryAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)

Instructions for updating:
Colocations handled automatically by placer.


c:\users\hyojong\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=128, output_dim=32, input_length=80, embeddings_regularizer=<keras.reg...)`
  import sys


Instructions for updating:
Use tf.cast instead.


c:\users\hyojong\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


Train on 7120 samples, validate on 880 samples
Epoch 1/5
7120/7120 [==============================] - 110s 15ms/step - loss: 0.4196 - accuracy: 0.8032 - binary_accuracy: 0.7061 - precision: 0.8136 - recall: 0.6455 - fmeasure: 0.6928 - val_loss: 0.2079 - val_accuracy: 0.9182 - val_binary_accuracy: 0.8103 - val_precision: 1.0000 - val_recall: 0.9185 - val_fmeasure: 0.9573
Epoch 2/5
7120/7120 [==============================] - 109s 15ms/step - loss: 0.1900 - accuracy: 0.9232 - binary_accuracy: 0.8449 - precision: 0.9236 - recall: 0.8989 - fmeasure: 0.9096 - val_loss: 0.2202 - val_accuracy: 0.9170 - val_binary_accuracy: 0.8681 - val_precision: 1.0000 - val_recall: 0.9174 - val_fmeasure: 0.9567
Epoch 3/5
7120/7120 [==============================] - 109s 15ms/step - loss: 0.1530 - accuracy: 0.9410 - binary_accuracy: 0.8817 - precision: 0.9391 - recall: 0.9281 - fmeasure: 0.9322 - val_loss: 0.1506 - val_accuracy: 0.9489 - val_binary_accuracy: 0.8925 - val_precision: 1.0000 - val_recall: 0.949

In [5]:
model_json = model.to_json()
with open("./saved_model/gru.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/gru.h5")